#To find the best SVM model


In [1]:
import pandas as pd
from sklearn import preprocessing
from IPython.display import display, HTML

df_loan200K=pd.read_csv('Data_Loans_200K.csv', header=0)
df_loan200K = df_loan200K.drop('Unnamed: 0', axis=1)

print(df_loan200K.shape)
cols_loan200K = df_loan200K.columns
 
    
print('Column Name, DataTypes, MissingValues in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any())

##If Yes, fill in missing values by mean values or most frequent nominal values.

df_loan200K["Credit Score"].fillna(df_loan200K["Credit Score"].mean(), inplace=True)
df_loan200K["Annual Income"].fillna(df_loan200K["Annual Income"].mean(), inplace=True)
df_loan200K["Years in current job"].fillna(df_loan200K["Years in current job"].mode().iloc[0], inplace=True)
df_loan200K["Months since last delinquent"].fillna(df_loan200K["Months since last delinquent"].mean(), inplace=True)
df_loan200K["Maximum Open Credit"].fillna(df_loan200K["Maximum Open Credit"].mean(), inplace=True)
df_loan200K["Bankruptcies"].fillna(df_loan200K["Bankruptcies"].mean(), inplace=True)
df_loan200K["Tax Liens"].fillna(df_loan200K["Tax Liens"].mean(), inplace=True)

print('\nColumn Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV\n')
for i in cols_loan200K:
    print(i,',', df_loan200K[i].dtype , ',', df_loan200K[i].isnull().any()) 
   
# encode labels
y = df_loan200K['Term'] # define label as nominal values
le = preprocessing.LabelEncoder()
le.fit(y)
y_encoded = le.transform(y) # encode nominal labels to integers #####################################
df_loan200K['Term'] = y_encoded

(200000, 17)
Column Name, DataTypes, MissingValues in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , True
Annual Income , float64 , True
Years in current job , object , True
Home Ownership , object , False
Purpose , object , False
Monthly Debt , float64 , False
Years of Credit History , float64 , False
Months since last delinquent , float64 , True
Number of Open Accounts , int64 , False
Number of Credit Problems , int64 , False
Current Credit Balance , int64 , False
Maximum Open Credit , float64 , True
Bankruptcies , float64 , True
Tax Liens , float64 , True

Column Name, DataTypes, MissingValues after filling with Mean and Frequent repeated nominal value in Loan 200K CSV

Loan Status , object , False
Current Loan Amount , int64 , False
Term , object , False
Credit Score , float64 , False
Annual Income , float64 , False
Years in current job , object , False
Home Ownership , object , False
Purpose , object ,

In [13]:
# Data preprocessing ################################################################################
print('Column Datatypes:\n',df_loan200K.dtypes)

# convert all nominal variables to binary variables
df_num=df_loan200K.copy(deep=True) 
# create new binary columns
df_dummies=pd.get_dummies(df_num[['Loan Status','Years in current job','Home Ownership','Purpose']])
# add them to dataframe
df_num=df_num.join(df_dummies)
# drop original columns
df_num=df_num.drop('Loan Status',axis=1)
df_num=df_num.drop('Years in current job',axis=1)
df_num=df_num.drop('Home Ownership', axis=1)
df_num=df_num.drop('Purpose', axis=1)

# drop extra binary columns, since we only need N-1 binary columns
df_num=df_num.drop('Loan Status_Charged Off', axis=1)
df_num=df_num.drop('Years in current job_10+ years', axis=1)
df_num=df_num.drop('Home Ownership_Own Home', axis=1)
df_num=df_num.drop('Purpose_wedding', axis=1)

# print out and display dataframe as table in HTML
display(HTML(df_num.head(10).to_html()))

# standarized data
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

scaler = StandardScaler()
df_num_std = df_num.copy(deep=True) 
x_features = df_num_std.loc[:, df_num_std.columns != 'Term']
cols = x_features.columns
df_num_std = pd.DataFrame(scaler.fit_transform(x_features), columns = cols)
df_num_std['Term'] = y_encoded
display('df_num_std:',HTML(df_num_std.head(10).to_html()))

# MinMax data
scaler = MinMaxScaler(feature_range=(1,5))
df_num_minmax = df_num.copy(deep=True) 
x_features = df_num_minmax.loc[:, df_num_std.columns != 'Term']
cols = x_features.columns
df_num_minmax = pd.DataFrame(scaler.fit_transform(x_features), columns = cols)
df_num_minmax['Term'] = y_encoded
display('df_num_minmax:',HTML(df_num_minmax.head(10).to_html()))

Column Datatypes:
 Loan Status                      object
Current Loan Amount               int64
Term                              int32
Credit Score                    float64
Annual Income                   float64
Years in current job             object
Home Ownership                   object
Purpose                          object
Monthly Debt                    float64
Years of Credit History         float64
Months since last delinquent    float64
Number of Open Accounts           int64
Number of Credit Problems         int64
Current Credit Balance            int64
Maximum Open Credit             float64
Bankruptcies                    float64
Tax Liens                       float64
dtype: object


,Current Loan Amount,Term,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Loan Status_Fully Paid,Years in current job_1 year,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation
0,445412,1,709.000000,1.167493e+06,5214.74,17.2,52.489525,6,1,228190,416746.0,1.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,262328,1,2476.680699,1.450412e+06,33295.98,21.1,8.000000,35,0,229976,850784.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,99999999,1,741.000000,2.231892e+06,29200.53,14.9,29.000000,18,1,297996,750090.0,0.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,347666,0,721.000000,8.069490e+05,8741.90,12.0,52.489525,9,0,256329,386958.0,0.0,0.0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,176220,1,2476.680699,1.450412e+06,20639.70,6.1,52.489525,15,0,253460,427174.0,0.0,0.0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,206602,1,7290.000000,8.968570e+05,16367.74,17.3,52.489525,6,0,215308,272448.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,217646,1,730.000000,1.184194e+06,10855.08,19.6,10.000000,13,1,122170,272052.0,1.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,648714,0,2476.680699,1.450412e+06,14806.13,8.2,8.000000,15,0,193306,864204.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,548746,1,678.000000,2.559110e+06,18660.28,22.6,33.000000,4,0,437171,555038.0,0.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
9,215952,1,739.000000,1.454735e+06,39277.75,13.9,52.489525,20,0,669560,1021460.0,0.0,0.0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


'df_num_std:'

,Current Loan Amount,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Loan Status_Fully Paid,Years in current job_1 year,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business,Purpose_vacation,Term
0,-0.241770,-6.711462e-01,-2.528999e-01,-1.078517,-0.500688,-2.256956e-16,-0.880003,-0.541782,-0.290817,-0.048597,-0.317035,-0.703764,0.756989,-0.281161,-0.309497,-0.300794,-0.279688,-0.285762,-0.274962,-0.273635,3.791926,-0.254689,-0.299933,-0.382873,1.316997,-0.712159,-0.200765,-0.189173,-0.197782,-0.857671,-0.180833,3.928766,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
1,-0.249662,1.726567e-16,2.081264e-16,1.156966,-0.199192,-1.413157e+00,1.174701,-0.766689,-0.286950,-0.007306,-0.760316,-0.703764,0.756989,-0.281161,-0.309497,-0.300794,-0.279688,-0.285762,-0.274962,-0.273635,-0.263718,-0.254689,-0.299933,-0.382873,1.316997,-0.712159,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
2,4.049796,-6.589966e-01,6.985621e-01,0.830937,-0.678494,-7.461172e-01,-0.029781,-0.541782,-0.139684,-0.016886,-0.760316,-0.703764,0.756989,-0.281161,-0.309497,-0.300794,-0.279688,-0.285762,-0.274962,-0.273635,3.791926,-0.254689,-0.299933,-0.382873,-0.759303,-0.712159,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
3,-0.245984,-6.665901e-01,-5.751887e-01,-0.797728,-0.902684,-2.256956e-16,-0.667448,-0.766689,-0.229895,-0.051431,-0.760316,-0.703764,0.756989,-0.281161,-0.309497,3.324537,-0.279688,-0.285762,-0.274962,-0.273635,-0.263718,-0.254689,-0.299933,-0.382873,-0.759303,-0.712159,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,0
4,-0.253374,1.726567e-16,2.081264e-16,0.149429,-1.358794,-2.256956e-16,-0.242336,-0.766689,-0.236106,-0.047605,-0.760316,-0.703764,0.756989,-0.281161,-0.309497,-0.300794,-0.279688,3.499416,-0.274962,-0.273635,-0.263718,-0.254689,-0.299933,-0.382873,-0.759303,1.404181,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
5,-0.252065,1.827503e+00,-4.948203e-01,-0.190652,-0.492958,-2.256956e-16,-0.880003,-0.766689,-0.318707,-0.062324,-0.760316,-0.703764,-1.321024,-0.281161,-0.309497,-0.300794,-0.279688,-0.285762,-0.274962,-0.273635,-0.263718,-0.254689,-0.299933,-0.382873,1.316997,-0.712159,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
6,-0.251588,-6.631730e-01,-2.379709e-01,-0.629503,-0.315152,-1.349630e+00,-0.384040,-0.541782,-0.520356,-0.062362,-0.317035,-0.703764,0.756989,-0.281161,-0.309497,-0.300794,-0.279688,-0.285762,-0.274962,-0.273635,-0.263718,-0.254689,3.334077,-0.382873,1.316997,-0.712159,-0.200765,-0.189173,-0.197782,1.165949,-0.180833,-0.254533,-0.193386,-0.222163,-0.190163,-0.185671,-0.181774,-0.257482,-0.178862,-0.182024,-0.182067,1
7,-0.233006,1.726567e-16,2.081264e-16,-0.314969,-1.196450,-1.413157e+00,-0.242336,-0.766689,-0.366343,-0.006030,-0

'df_num_minmax:'

,Current Loan Amount,Term,Credit Score,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens,Loan Status_Fully Paid,Years in current job_1 year,Years in current job_2 years,Years in current job_3 years,Years in current job_4 years,Years in current job_5 years,Years in current job_6 years,Years in current job_7 years,Years in current job_8 years,Years in current job_9 years,Years in current job_< 1 year,Home Ownership_HaveMortgage,Home Ownership_Home Mortgage,Home Ownership_Rent,Purpose_Business Loan,Purpose_Buy House,Purpose_Buy a Car,Purpose_Debt Consolidation,Purpose_Educational Expenses,Purpose_Home Improvements,Purpose_Medical Bills,Purpose_Other,Purpose_Take a Trip,Purpose_major_purchase,Purpose_moving,Purpose_other,Purpose_renewable_energy,Purpose_small_business
0,1.017386,1,1.071625,1.026368,1.047859,1.813154,2.192944,1.315789,1.266667,1.027761,1.001083,1.571429,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.010062,1,2.092668,1.033207,1.305578,2.046338,1.181818,2.842105,1.000000,1.027978,1.002210,1.000000,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,5.000000,1,1.090108,1.052097,1.267991,1.675635,1.659091,1.947368,1.266667,1.036254,1.001949,1.000000,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.013476,0,1.078556,1.017653,1.080230,1.502242,2.192944,1.473684,1.000000,1.031185,1.001005,1.000000,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.006617,1,2.092668,1.033207,1.189423,1.149477,2.192944,1.789474,1.000000,1.030836,1.001110,1.000000,1.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.007833,1,4.872924,1.019827,1.150217,1.819133,2.192944,1.315789,1.000000,1.026194,1.000708,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
6,1.008275,1,1.083755,1.026772,1.099624,1.956652,1.227273,1.684211,1.266667,1.014863,1.000707,1.571429,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7,1.025519,0,2.092668,1.033207,1.135885,1.275037,1.181818,1.789474,1.000000,1.023517,1.002245,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
8,1.021520,1,1.053718,1.060007,1.171257,2.136024,1.750000,1.210526,1.000000,1.053185,1.001442,1.000000,1.0,5.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
9,1.008207,1,1.088953,1.033312,1.360476,1.615845,2.192944,2.052632,1.000000,1.081458,1.002654,1.000000,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,5.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [3]:
# 02. Logistic regression

import pandas as pd
import numpy as np
from sklearn import metrics 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, precision_score, accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix as cm

x=df_num.drop('Term',axis=1)
y=df_num['Term']

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
clf=LogisticRegression(penalty='l2',solver='lbfgs')
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)

confM = cm(y_test, y_pred)
print(confM)

acc=accuracy_score(y_test, y_pred)
pre=precision_score(y_test, y_pred, average='macro')
print('By hold-out evaluation: acc = ',acc, ', precision = ', pre)

[[  171 19362]
 [  171 30296]]
By hold-out evaluation: acc =  0.60934 , precision =  0.5550465181843811


In [4]:
# 03. SVM

from sklearn.svm import SVC

# by hold-out evaluation
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

clf=SVC(kernel='linear', C=0.0000000001, max_iter=1) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=0.0000000001, max_iter=1) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.38594
Accuracy by hold-out evaluation:  0.38614


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [5]:
clf=SVC(kernel='linear', C=0.0000000001, max_iter=10) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=0.0000000001, max_iter=10) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57822
Accuracy by hold-out evaluation:  0.57778


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=10).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [6]:
clf=SVC(kernel='linear', C=0.0000000001, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=0.0000000001, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.6139


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57766


In [8]:
clf=SVC(kernel='linear', C=1, max_iter=1) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=1, max_iter=1) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.41448
Accuracy by hold-out evaluation:  0.38616


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [9]:
clf=SVC(kernel='linear', C=1, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=1, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.5783


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57768


In [10]:
clf=SVC(kernel='linear', C=100, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=100, max_iter=100) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.5783


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57748


In [11]:
clf=SVC(kernel='linear', C=100, max_iter=20000) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=100, max_iter=20000) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57822


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57732


In [12]:
clf=SVC(kernel='linear', C=1000, max_iter=20000) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)
clf=SVC(kernel='rbf', C=1000, max_iter=20000) 
clf=clf.fit(x_train, y_train)
y_pred=clf.predict(x_test)
acc=accuracy_score(y_test, y_pred)
print('Accuracy by hold-out evaluation: ',acc)

C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57824


C:\Users\vikas\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy by hold-out evaluation:  0.57728
